# Introduction to text analysis VIII #

## topic detection ##

Topic detection is a way to extract relevant information from texts. The topic is a set of words (from the text) having particular relevance in terms of probability. They apper to be words that characterize the topics (one or more) discussed in the documents.

**definitions:**

* Document: A single text, paragraph or even tweet to be classified
* Word/Term: a single component of a document
* Topic: a set of words describing a group (cluster) of documents

** each document usually is as a mixture of several topics **

### mixture of topics ###

A model such as LDA will produce an classification such as the following:

* Sentences 1 and 2: 100% Topic A
* Sentences 3 and 4: 100% Topic B
* Sentence 5: 60% Topic A, 40% Topic B

Topic A: 30% broccoli, 15% bananas, 10% breakfast, 10% munching, … (at which point, you could interpret topic A to be about food)

Topic B: 20% chinchillas, 20% kittens, 20% cute, 15% hamster, … (at which point, you could interpret topic B to be about cute animals)

### methodologies ###

* latent dirichlet allocation (lda) 
* Non negative matrix factorization
* Clustering 

**latent dirichlet allocation (lda)** 

It's a complex mathematical model (based on Bayesian statistics and Dirichlet and Multinomial distributions) to establish the words in a set of documents that are the most representative. The starting point is definining a 

* fixed number of topics K
* to each topic k we associate a probability p = p(k|w) i.e. the probability of seeing the topic k given the set of words w in the document d
* to each topic k we associate a probability s = s(k,d) i.e. the probability of a k topic belonging to the document d. The distribution s represents the mixture of topics related to d
* A word in the document is picked by randomly extracting from a topic and from a document according to s and p distributions
* An optimization is performed fitting the s,p distributions to the actual distribution of words in the documents.

**Non negative matrix factorization **

![title](NMF.png)

* **V**  is the matrix representing all documents
* **H** is the matrix representing documents given the topics
* **W** is the matrix representing the topics


the factorization is made using objective functions such as *Frobenius Norm *

### Main features ###

** LDA **

* Slow method
* Quite accurate for large corpora where each document is a mixture of topics
* Most adopted 

** NMF **

* Fast method
* Accurate with small corpora (i.e. tweets) or tweets with no mixture of topics
* not commonly adopted


## hands on ##

In [1]:
import string
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [3]:
from sklearn.datasets import fetch_20newsgroups

** get the corpus **

20 newsgroup

In [4]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print(len(docs_raw))

11314


In [5]:
stops_it = stopwords.words('italian')
stops_en = stopwords.words('english')

translator = str.maketrans(' ', ' ', string.punctuation) ## remove the punctuation

In [6]:
def minimumSize(tokens,llen = 2):
    ## remove words smaller than llen chars
    tks = []
    for t in tokens:
        if(len(t) > llen):
            tks.append(t)
    return tks

def removeStops(tokens,stops = stops_it):
    # remove stop words
    remains = []
    for t in tokens:
        if(t not in stops):
            remains.append(t)
    return remains

def processText(text):
    ## tokenizer with stop words removal and minimum size 
    tks = word_tokenize(text)
    tks = [t.translate(translator) for t in tks] ## remove the punctuation
    tks = minimumSize(tks)
    tks = removeStops(tks,stops_en)
    return tks

### TFIDF vectorizer ###

It transforms each word in the D documents in a sparse matrix representing a normalized frequency of each word in each document. 

In [7]:
n_features = 1000 
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,tokenizer=processText)

**n_features** it's the number of individual ters from the corpus to use (notice that rarely a language by humans uses more than few thousands of distinct words ). Having a large dataset it is safe to use large number for n_features, for short corpus n_features must be non large

**max_df** is the probability at which the more probable words must be removed (removes the most common words)

**min_df** removes the words appearing less than 2 times in the dataset.


In [8]:
corpusT = docs_raw[0:500] ## let's use the first 500 documents

tfidf = tfidf_vectorizer.fit_transform(corpusT)

In [9]:
tfidf

<500x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 12953 stored elements in Compressed Sparse Row format>

** associate names (words) to each feature **

In [10]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

** LDA **

In [11]:
n_topics = 20
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=10,
                                learning_method = 'batch')


* ** n_topics ** is somehow arbitrary. 
* ** max_iter ** stops the iteration after maximum 10
* ** learning method ** is usually online but can be also batch (slower) when all data are processed at time

In [12]:
lda.fit(tfidf) ## fit the model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [13]:
def mostImportantWordsPerTopic(feature_names,topic,n_top_words):
    mwords = []
    sort_topic = topic.argsort()
    mw = sort_topic[:-n_top_words - 1:-1] ## reversed list    
    for idx in mw:
        mwords.append(feature_names[idx])
    return mwords
        

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        
        most_important_words = mostImportantWordsPerTopic(feature_names,topic,n_top_words)

        message = "Topic #%d: " % topic_idx
        message += " ".join(most_important_words)
        print(message)
    print()

** Printing the topics **

In [14]:
n_top_words = 10
print_top_words(lda, tfidf_feature_names, n_top_words)

Topic #0: clipper board thank required logic image anybody display left thanks
Topic #1: people one would know think may time even want anyone
Topic #2: cause post following group drugs based asking armenian several bbs
Topic #3: keys advance package red referred software mac gear object plus
Topic #4: god received message post neither hell heaven contact require follow
Topic #5: spacecraft soon tower thanks net manager love available legal jesus
Topic #6: problems printer change course obvious file shuttle software murray anyone
Topic #7: anything nature gun development weeks children death hello jesus believe
Topic #8: battery child car print files sense charge scope pro possibly
Topic #9: national ask wish knows excuse circuit anybody program reserve audio
Topic #10: use would like please email get also need thanks anyone
Topic #11: drug april worked land program occurs launched asking marijuana among
Topic #12: water motif starters technology killed israel cases sale morality great

** NMF **

In [15]:
nmf = NMF(n_components=n_topics, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

** parameters **

NMF is basically free of parameters :).
    
* alpha : regolarization parameter (used to smooth the frequencies and to improve the fit)
* l1_ratio : regolarization parameter (used to smooth the frequencies and to improve the fit)

In [16]:
n_top_words = 10
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topic #0: would get one think people like know much could may
Topic #1: thanks please email address list advance information available net anybody
Topic #2: use simms memory machine mac could several need answer work
Topic #3: year last yes old three years great game time mask
Topic #4: problem found used light known check error however think running
Topic #5: file printer print manager like another port instead name driver
Topic #6: window box control want left get option application manager upper
Topic #7: looking card working must email condition mail appreciated buy spend
Topic #8: problems pain obvious gave anybody following ask sure also cars
Topic #9: possible yes phone crypto interest invalid fire eternal understanding soviet
Topic #10: things apparently worse like also little exactly seem basically reality
Topic #11: post message product real research feel could server sorry error
Topic #12: program windows files april run software microsoft image code version
Topic #13: lost 

### Bonus visualization ###

In [17]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [18]:
import warnings
warnings.filterwarnings('ignore')

** default visualization of topics and frequency in a multidimensional space **

In [19]:
pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      29.285331        1       1 -0.270135  0.066481
10     18.329528        1       2 -0.044943 -0.267703
7       6.137466        1       3  0.065051  0.055765
2       5.483792        1       4  0.055935  0.002267
19      4.517869        1       5  0.065437  0.015143
13      3.424429        1       6  0.010159  0.035876
3       3.116749        1       7  0.014326  0.007817
18      2.843758        1       8  0.035165  0.015423
16      2.702528        1       9  0.009537  0.004433
14      2.598517        1      10  0.008906 -0.000669
9       2.523051        1      11  0.007146  0.005007
12      2.496457        1      12  0.016266  0.008297
0       2.488038        1      13  0.005742  0.012835
8       2.399080        1      14  0.013667  0.002499
15      2.333220        1      15  0.025009  0.000847
17      2.129968        1      16  0.003028  0.012335
4       1.963268        1      17  0.008796  0.001251
6       1.896590        1      18 -0.009232 -0.000602
5       1.857812        1      19 -0.010954  0.011904
11      1.472545        1      20 -0.008909  0.010794, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
673   Default   3.000000     problems   3.000000  30.0000  30.0000
876   Default   7.000000       thanks   7.000000  29.0000  29.0000
881   Default   5.000000       things   5.000000  28.0000  28.0000
655   Default   3.000000     possible   3.000000  27.0000  27.0000
929   Default   8.000000          use   8.000000  26.0000  26.0000
676   Default   4.000000      program   4.000000  25.0000  25.0000
101   Default   3.000000      believe   3.000000  24.0000  24.0000
355   Default   3.000000          god   3.000000  23.0000  23.0000
37    Default   6.000000         also   6.000000  22.0000  22.0000
672   Default   6.000000      problem   6.000000  21.0000  21.0000
490   Default   4.000000      looking   4.000000  20.0000  20.0000
571   Default   5.000000         need   5.000000  19.0000  19.0000
794   Default   2.000000        simms   2.000000  18.0000  18.0000
206   Default   7.000000        could   7.000000  17.0000  17.0000
324   Default   4.000000        first   4.000000  16.0000  16.0000
422   Default   4.000000  information   4.000000  15.0000  15.0000
987   Default  12.000000        would  12.000000  14.0000  14.0000
657   Default   3.000000         post   3.000000  13.0000  13.0000
209   Default   4.000000       course   4.000000  12.0000  12.0000
108   Default   2.000000         bike   2.000000  11.0000  11.0000
873   Default   1.000000         test   1.000000  10.0000  10.0000
994   Default   4.000000         year   4.000000   9.0000   9.0000
897   Default   2.000000        today   2.000000   8.0000   8.0000
971   Default   2.000000      windows   2.000000   7.0000   7.0000
953   Default   2.000000        water   2.000000   6.0000   6.0000
317   Default   2.000000         file   2.000000   5.0000   5.0000
481   Default   2.000000         list   2.000000   4.0000   4.0000
403   Default   4.000000      however   4.000000   3.0000   3.0000
530   Default   2.000000       memory   2.000000   2.0000   2.0000
48    Default   8.000000       anyone   8.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
463   Topic20   0.235527     launched   0.972362   2.8003  -4.8048
514   Topic20   0.212465    marijuana   0.959269   2.7108  -4.9078
981   Topic20   0.322463       worked   1.497079   2.6829  -4.4906
42    Topic20   0.190694        among   1.007280   2.5538  -5.0159
544   Topic20   0.172651      mission   0.940430   2.5231  -5.1153
761   Topic20   0.180078        scope   1.109443   2.4000  -5.0732
456   Topic20   0.290512         land   2.044712   2.2668  -4.5950
433   Topic20   0.140535     involved   1.015140   2.2409  -5.3211
857   Topic20   0.1232

** another decomposition method **

In [20]:
pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer,mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      49.999938        1       1 -0.112187 -0.062988
14      5.291949        1       2 -0.431058  0.040370
3       4.712628        1       3 -0.000457 -0.269504
1       4.692684        1       4 -0.223809 -0.379263
2       4.200966        1       5  0.065014  0.346491
4       3.074577        1       6 -0.181561  0.202078
12      3.005565        1       7 -0.404867  0.220240
7       2.976061        1       8 -0.354299 -0.297598
18      2.637333        1       9 -0.028071 -0.500952
10      2.411257        1      10  0.364070 -0.348514
13      2.371424        1      11  0.186905 -0.418481
9       2.052399        1      12  0.273732  0.438734
11      2.002994        1      13 -0.478395 -0.142764
19      1.803490        1      14  0.059269  0.113978
5       1.753592        1      15 -0.282969  0.400093
8       1.544231        1      16 -0.060911  0.513758
16      1.405282        1      17  0.272799 -0.078708
6       1.397662        1      18  0.450246 -0.118293
17      1.367737        1      19  0.501754  0.094644
15      1.298231        1      20  0.384796  0.246679, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
655   Default  26.000000   possible  26.000000  30.0000  30.0000
881   Default  23.000000     things  23.000000  29.0000  29.0000
929   Default  28.000000        use  28.000000  28.0000  28.0000
672   Default  21.000000    problem  21.000000  27.0000  27.0000
48    Default  24.000000     anyone  24.000000  26.0000  26.0000
673   Default  16.000000   problems  16.000000  25.0000  25.0000
994   Default  17.000000       year  17.000000  24.0000  24.0000
876   Default  19.000000     thanks  19.000000  23.0000  23.0000
657   Default  13.000000       post  13.000000  22.0000  22.0000
646   Default  14.000000     please  14.000000  21.0000  21.0000
996   Default  13.000000        yes  13.000000  20.0000  20.0000
882   Default  17.000000      think  17.000000  19.0000  19.0000
490   Default  11.000000    looking  11.000000  18.0000  18.0000
794   Default  12.000000      simms  12.000000  17.0000  17.0000
676   Default  11.000000    program  11.000000  16.0000  16.0000
272   Default  13.000000      email  13.000000  15.0000  15.0000
987   Default  23.000000      would  23.000000  14.0000  14.0000
773   Default   9.000000       send   9.000000  13.0000  13.0000
577   Default  12.000000        new  12.000000  12.0000  12.0000
493   Default   8.000000       lost   8.000000  11.0000  11.0000
476   Default  14.000000       like  14.000000  10.0000  10.0000
317   Default   7.000000       file   7.000000   9.0000   9.0000
947   Default  10.000000       want  10.000000   8.0000   8.0000
450   Default  15.000000       know  15.000000   7.0000   7.0000
348   Default  13.000000        get  13.000000   6.0000   6.0000
456   Default   6.000000       land   6.000000   5.0000   5.0000
970   Default   6.000000     window   6.000000   4.0000   4.0000
630   Default  10.000000     people  10.000000   3.0000   3.0000
953   Default   6.000000      water   6.000000   2.0000   2.0000
510   Default   8.000000        man   8.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
981   Topic20   0.920441     worked   1.344739   3.9651  -3.3060
684   Topic20   0.614900    purpose   0.946497   3.9129  -3.7094
805   Topic20   0.521638       sold   0.970468   3.7234  -3.8739
225   Topic20   0.457632       deal   0.915508   3.6508  -4.0048
900   Topic20   0.458716       took   1.105211   3.4648  -4.0024
484   Topic20   0.440364     living   1.102349   3.4266  -4.0432
374   Topic20   0.156798   happened   0.414927   3.3710  -5.0759
119   Topic20   0.494434     bought   1.308914   3.3706  -3.9274
469   Topic20   0.506488      legal   1.495986   3.2611  -3.9033
660   Topic20   1.142537      power   3